In [8]:
import overpy
import pandas as pd

# Initialisiere die Overpass API mit einer benutzerdefinierten URL
api = overpy.Overpass(url="http://overpass.osm.ch/api/interpreter")

# Overpass Query für Wanderwege innerhalb der Schweiz. Mit Center erhalten wir die Koordinaten in der Mitte eines Wnaderweges
query = """
[out:json];
relation
["route"="hiking"]
/*["name"]*/
["name"!~"fixme", i]
["network"="lwn"]
["osmc:symbol"~"yellow::yellow_diamond|red:white:red_bar|yellow:white:yellow_diamond|blue:white:blue_bar"]
(id: 2408854)
(45.8899, 6.0872, 47.8085, 10.4921);
out center tags;
"""

# Führe die Anfrage aus
result = api.query(query)

# Liste zum Speichern der extrahierten Informationen
list = []

# Iteration über alle relations
for relation in result.relations:
    
    # Extrahiere die relevanten Daten

    org_name = relation.tags.get('name')
    fix_name = ""
    org_to = relation.tags.get('to')
    org_from = relation.tags.get('from')
    
    # Center ist ein Tupel mit langitute und longitude, wir möchten nur einen wert
    lat = getattr(relation, 'center_lat')
    lon = getattr(relation, 'center_lon')
    
    # Wenn der orginal Name nicht vorhanden ist setze ihn aus from und to zusammen
    if not org_name and org_from and org_to :
        fix_name = f"{org_from} - {org_to}"
    else:
        fix_name = org_name

    # Erstelle ein Dictionary um die Attribute als Tupel zu speichern
    if fix_name and lat > 0 and lon > 0: 
        dict = {
        'id': relation.id,
        'name': fix_name,
        'lat': lat,
        'lon': lon
        }

        # Jedes Tupel wird nun in der Liste als neue Zeile gespeichert
        list.append(dict)

# Wenn alle Daten verarbeitet wurden, erstelle das DataFrame
df = pd.DataFrame(list)

# Zeige das DataFrame an
print(df.lat)


0    45.8988159
Name: lat, dtype: object


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Verbindung zur MySQL-Datenbank herstellen (ersetze Platzhalter mit deinen Daten)
user = 'root'        # MySQL-Benutzername
password = ''  # MySQL-Passwort
host = '127.0.0.1'   # Server (bei lokalen Installationen ist das oft 'localhost')
database = 'mydatabase'  # Name der Datenbank, in die du schreiben möchtest

# Erstelle die Verbindungs-Engine
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

# DataFrame in die MySQL-Datenbank-Tabelle schreiben (wenn die Tabelle nicht existiert, wird sie erstellt)
df.to_sql(name='Wanderwege', con=engine, if_exists='replace', index=False)

print("DataFrame erfolgreich in die MySQL-Datenbank geladen!")

In [3]:
# Import required libraries
import numpy as np
import pandas as pd

import openmeteo_requests
import requests_cache
from retry_requests import retry

import matplotlib.pyplot as plt
import seaborn as sns

## Use command "pip freeze > requirements.txt" to save the dependencies in requirements.txt

In [ ]:
# Create a pandas dataframe with coordinates of locations to process
# Locations and coordinates found in openStreetMap
locations_coords = pd.DataFrame({
    "location" : ["Schönengrund, Hinterland, Appenzell Ausserrhoden, 9105, Switzerland", 
              "Einsiedeln, Schwyz, 8840, Switzerland",
              "Wolfenschiessen, Nidwalden, Switzerland",
              "Pontresina, Maloja, Grisons, 7504, Switzerland",
              "Ruvigliana, Circolo di Lugano est, Lugano, Distretto di Lugano, Ticino, 6977, Switzerland"
              ], 
    "lat" : [47.31706, 47.1071, 46.9062, 46.4774, 46.00865], 
    "lon" : [9.24482, 8.8114, 8.4282, 9.9042, 8.98767]})
print(locations_coords)


                                            location       lat      lon
0  Schönengrund, Hinterland, Appenzell Ausserrhod...  47.31706  9.24482
1              Einsiedeln, Schwyz, 8840, Switzerland  47.10710  8.81140
2            Wolfenschiessen, Nidwalden, Switzerland  46.90620  8.42820
3     Pontresina, Maloja, Grisons, 7504, Switzerland  46.47740  9.90420
4  Ruvigliana, Circolo di Lugano est, Lugano, Dis...  46.00865  8.98767


In [ ]:
print(type(list(locations_coords.loc[:, "location"])))
print(list(locations_coords.loc[:, "location"]))

<class 'list'>
['Schönengrund, Hinterland, Appenzell Ausserrhoden, 9105, Switzerland', 'Einsiedeln, Schwyz, 8840, Switzerland', 'Wolfenschiessen, Nidwalden, Switzerland', 'Pontresina, Maloja, Grisons, 7504, Switzerland', 'Ruvigliana, Circolo di Lugano est, Lugano, Distretto di Lugano, Ticino, 6977, Switzerland']


In [4]:
# Define a start and end date of data to fetch from Open-Meteo API
start_date = "2024-01-01"
end_date = "2024-09-09"

In [5]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)     # stores data for 1 hour in the cache
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)         # retries 5 times with increasing delay of 20%
openmeteo = openmeteo_requests.Client(session = retry_session)                  # Using the retry_session to make a request to Open-Meteo API

In [16]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
params = {
#	"latitude": list(df["lat"]),
#	"longitude": list(df["lon"]),
	"latitude": df.lat,
	"longitude": df.lon,
	"start_date": start_date,
	"end_date": end_date,
	"hourly": ["temperature_2m", "relative_humidity_2m", "precipitation_probability", "precipitation", "rain", "showers", "snowfall", "snow_depth", "weather_code", "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "visibility", "wind_speed_10m", "wind_direction_10m", "wind_gusts_10m", "uv_index", "uv_index_clear_sky", "is_day", "sunshine_duration"],
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "apparent_temperature_max", "apparent_temperature_min", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "uv_index_max", "uv_index_clear_sky_max", "precipitation_sum", "rain_sum", "showers_sum", "snowfall_sum", "precipitation_hours", "precipitation_probability_max", "wind_speed_10m_max", "wind_gusts_10m_max"],
	"timezone": "Europe/Berlin"
}
responses = openmeteo.weather_api(url, params=params)
#	 print(f"Number of resposes: {len(responses)}")
print (df.lat)


0    45.8988159
Name: lat, dtype: object


In [15]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 45.900001525878906°N 7.159999370574951°E
Elevation 2570.0 m asl
Timezone b'Europe/Berlin' b'CEST'
Timezone difference to GMT+0 7200 s
